<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
<b> This notebook walks us through how to compute a tuning curve using the Allen Brain Observatory data</b>


This notebook has two parts.  The first walks through computing a tuning curves for cells in response to a drifting grating stimulus. This is a classic single-cell analysis, and students can compare the tuning of several cells from the same experiment. The second part looks at correlations between cells that have the same tuning to the drifting grating stimulus from the same experiment. This highlights the fact that our dataset has populations of cells, simultaneously imaged, and allows for examinations of how those cells interact with each other.
This notebook is designed to only reference a single experiment from the Allen Brain Observatory, thus requires only one NWB file to be downloaded. (as downloading NWB files can be time consuming).

### Standard Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import sys, os

### Brain Observatory set up

In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
#if you have put the brain observatory folder somewhere specific (rather than the default), 
#you'll need to point the Cache to the manifest_file. Otherwise, you should be set to go!

boc = BrainObservatoryCache()

## Computing tuning curve for the drifting grating stimulus

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>We need:

<li>fluorescence trace for our cell.  We will use the DFF trace
<li>stimulus information for the drifting grating stimulus

</div>

In [ ]:
cell_id = 541513979

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
I've created a function here that will return the DF/F trace and the stimulus table if we provide a cell id and a stimulus name. This function leverages functions in the AllenSDK, and there are examples that walk through these steps in other notebooks on the SDK page. I'm happy to explain the steps in greater detail if anyone is interested.  
<p>The key things to know about this.  You provide the function with a cell_specimen_id and a stimulus name.  It returns:
<li>the timestamps for the DF/F trace.  This is a numpy array.
<li>the trace of that cell's DF/F trace for the whole session.  This is also a numpy array.
<li>a table that describes the stimulus conditions and timing.  This is a <b>pandas</b> dataframe.
<p> If you aren't working in AWS, this function can take some time the first time you run it for a given NWB file as it will require that file to be downloaded from the warehouse.
</div>

In [ ]:
def get_dff_traces_and_stim_table(cell_specimen_id, stimulus): 
    #identify the session for a given cell id and stimulus
    exps = boc.get_ophys_experiments(cell_specimen_ids=[cell_specimen_id], stimuli=[stimulus])
    #get the session_id for that session
    session_id = exps[0]['id']
    #access the data for that session
    data_set = boc.get_ophys_experiment_data(session_id)
    #get the DFF trace for the cell
    timestamps, dff = data_set.get_dff_traces(cell_specimen_ids=[cell_specimen_id])
    dff_trace = dff[0,:]
    #get the stimulus table for the stimulus
    stim_table = data_set.get_stimulus_table(stimulus)
    #return everything
    return (timestamps, dff_trace, stim_table)

In [ ]:
timestamps, dff_trace, stim_table = get_dff_traces_and_stim_table(cell_id, 'drifting_gratings')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
First let's plot the DF/F trace of our cell to see what it looks like
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
Now let's look at our stimulus table to see what information we have. We just want to see the first few lines, so use the function <b>head</b> to see the top of this DataFrame.
</div>

In [ ]:
stim_table.head()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
The stimulus table has 5 columns.  Start and end indicate the <b>frame number</b> when a given grating condition starts and ends, respectively. The other columns indicate what the grating codition is, including the temporal frequency of the grating (in Hz), the direction (called orientation) of the grating (in degrees), and whether the grating is a blank sweep (eg. a gray screen). 
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
Pandas is a very useful python module for data analysis, which has an object called a <b>DataFrame</b> that is a flexible and powerful tool for analyzing large datasets. I highly encourage interested students to explore this analysis module. But for our purposes, we will only use it to access the stimulus information. 
</div>

## Pandas

<div style="border-radius: 3px; padding: 10px;  background: #F0FAFF; ">
<b>Quick pandas tutorial for our purposes today!</b>
<p>
To access data from a DataFrame we must specify the column we are using and specify the row using the <b>index</b>. To specify a column we can use two methods:
<li> stim_table['start']
<li> stim_table.start
<p> Then to specify the row we want we must use the index of that row. 
<li> stim_table['start'][0]
<li> stim_table.start[0]
<p> We can also subselect portions of the DataFrame using the values in the DataFrame. For example, to select only the rows of the table where the orientation is 90 degrees we can use:
<li> stim_table[stim_table.orientation==90]
<p>Try this yourself.  Note the index.  The rows of this subselected DataFrame maintain the indices of the original DataFrame. Now in order to get a specific row, you either need to know it's original index, or use <b>iloc</b>. For example, this will return the first row of the subselected DataFrame regardless of the original index of that row:
<li> stim_table[stim_table.orientation==90].iloc[0]
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
To look at the cell's response to a given grating presentation, let's plot the DF/F of the cell during the presentation of that grating.  We want to pad the plot with ~ 1 second of the DF/F trace preceding the grating presentation.  1 second = 30 frames.  We'll plot the response to the first grating presentation.
<div>

In [ ]:
plt.plot(dff_trace[stim_table.start[0]-30:stim_table.end[0]+30])
plt.axvspan(30,90, color='gray', alpha=0.3) #this shades the period when the stimulus is being presented
plt.ylabel("DF/F")
plt.xlabel("Frames")

We want to quantify this response. We can explore different methods of quantifying this.
* mean DF/F during the grating presentation
* sum of the DF/F during the grating presentation (are these different?)
* maximum DF/F during grating

For now let's use the mean DF/F during the presentation of the grating.

In [ ]:
dff_trace[stim_table.start[0]:stim_table.end[0]].mean()

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 1:** Repeat this for the next grating stimulus </div>

<div style="background: #eab6f0; border-radius: 3px; padding: 10px;">
    <b>Breakout 1</b>
    </div

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

Already we can see that some stimulus conditions elicit larger responses than others.  This is what we want to quantify. 

<p><p>To do this, let's calculate the mean DF/F for each grating presentation in this stimulus. To start, let's create a numpy array to hold our calculated responses. We'll have three columns, one for the stimulus orientation, one for temporal frequency, and the last for the response. Then we need to iterate over all stimulus trials, populate the orientation and TF and then calculate the mean response.
    
</div>

In [ ]:
cell_response= np.zeros((len(stim_table),3))
for i in range(len(stim_table)):
    cell_response[i,0] = stim_table.orientation[i]
    cell_response[i,1] = stim_table.temporal_frequency[i]
    cell_response[i,2] = 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
If we only care about one stimulus parameter, we can quickly compare the response to that parameter, say the direction. Here we will plot each grating response as a function of the grating orientation.
</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 2:** Repeat this for the temporal frequency parameter </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
We want to quantify this more explicitly.  So let's average all of the responses to each orientation together. This is the mean DF/F response to an orientation, for all temporal frequencies, for all trials.  For example, for orientation=270: (Hint: use np.where)
</div>

In [ ]:
trials = np.where(cell_response[:,0]==270)[0]
cell_response[trials,2].mean()

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 3:** Compute and plot the mean response as a function of orientation
<p> To start, you need to know what all the possible orientation values are. You can either find this from the website, or you can find the <b>unique</b> values that are not NaNs (eg. values that are <b>finite</b>)
</div>

In [ ]:
all_ori = np.unique(cell_response[:,0])
orivals = all_ori[np.isfinite(all_ori)]
print(orivals)

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 4:** Compute and plot the mean response as a function of temporal frequency for all orientations. </div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 5:** Add errorbars to the above tuning curves. They can be standard deviation or standard error or the mean.  (Hint: plt.errorbar might be a useful function).
</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
**Exercise 6:** Add a black line showing the mean response to the blank sweep (Hint 1: orientation and temporal frequency are NaN for the blank sweep condition.  Hint 2: plt.axhline might be a useful function).  </div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 7:** Compute and plot the direction tuning curve separately for each of the 5 temporal frequencies. </div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 8:** Are there other ways to visualize these tuning responses? </div>

<div style="background: #eab6f0; border-radius: 3px; padding: 10px;">
    <b>Breakout 2</b>
    </div

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
We've looked at the tuning curve for one cell. There are a lot of other cells in this experiment. Now we're going to see what some of the other cells in this experiment look like, and start the explore how the different cells might interact.

</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 9:** Compute the tuning curves for cell_ids 541512490, 541512611, 541512645, 541512079, 541511403, 541511670, 541511373, 541513771, 541511385, 541512607. (Hint: it might be helpful to write a function) In what ways do these tuning curves differ? In what ways are they the same? What are interesting parameters of a cell's response to this stimulus?  </div>

In [ ]:
def compute_ori_tf_tuning(cell_id):
    
    return tuning_array

In [ ]:
tuning_a = compute_ori_tf_tuning(541512490)
plt.imshow(tuning_a)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
Some of these cells show the same tuning for orientation and temporal frequency. Now let's look at these more closely to see how they are <i>correlated</i>.

<p>If you've overwritten the dff_trace, go back and get the trace for our original cell: cell_id = 541513979
<p>Also get the trace for cell 541511905, let's call this trace "dff_trace_2"
</div style>

In [ ]:
cell_id = 541513979
timestamps, dff_trace, stim_table = get_dff_traces_and_stim_table(cell_id, 'drifting_gratings')

In [ ]:
new_cell_id = 541511905
timestamps, dff_trace_2, stim_table = get_dff_traces_and_stim_table(new_cell_id, 'drifting_gratings')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
These two cells have some similarities in their tuning preferences, but how similar are their responses?
</div>

<div style="border-radius: 3px; padding: 10px;  background: #F0FAFF; ">
<b>Correlations</b>
<p>
We can compute a correlation coefficient to measure the relationship between two variables - in this case the responses of two different cells. When cell A has a large response, does cell B also have a strong response? 
<p> We will use Pearson correlation coefficient, which is defined as:
<img src="correlation_coefficient_formula.svg">
<p> where x and y are the two variables in question. We will use a function from scipy.stats to compute this for us.
<p> If the two variables are positively correlated, a large response from cell A will be matched with a large response in cell B. If they are weakly correlated, the response size of cell A will have not relationship with the response size of cell B. And if they are negatively correlated, cell B will have small responses when cell A has large responses.
<img src="pearson-2-small.png">
</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 10:** Compute the correlation between these two cells' traces. We're going to use st.pearsonr from scipy.stats
</div>

In [ ]:
import scipy.stats as st

In [ ]:
r,p = st.pearsonr(dff_trace, dff_trace_2)
print(r)

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 11:** Calculate the response of the second cell to all the stimulus trials and compute the correlation between these two cells' responses.
</div>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 12:** Calculate the orientation tuning curve for the second cell and compute the correlation between these two cells' tuning. Hint: you will need to <b>flatten</b> the tuning array.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
We've looked at three different correlations. Discuss why these correlations are different and what it might tell you about these two cells.
    
</div style>

<div style="background: #FFF0F0; border-radius: 3px; padding: 10px;">
<p>**Exercise 13:** Repeat these three correlation computations for cells 541511373 and 541511385. Notice anything similar or different from the previous pair?
</div>

In [ ]:
cell_id = 541511373
timestamps, dff_trace_3, stim_table = get_dff_traces_and_stim_table(cell_id, 'drifting_gratings')
cell_id = 541511385
timestamps, dff_trace_4, stim_table = get_dff_traces_and_stim_table(cell_id, 'drifting_gratings')


<div style="background: #eab6f0; border-radius: 3px; padding: 10px;">
    <b>Breakout 3</b>
    </div

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

Everything so far has looked at the responses of these cells to 'driftings_gratings'. Now let's look at their responses to 'natural_movie_three' and see how it compares to their grating responses.

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
Use the function from the start of this notebook to get the timestamps, dff trace, and stimulus table for our original cell (cell_id = 541513979) for 'natural_movie_three'. Look at the stimulus table. How is it different from the stimulus table for drifting gratings?
</div style>

In [ ]:
cell_id = 541513979
timestamps, dff_trace, stim_table_nm = get_dff_traces_and_stim_table(cell_id, 'natural_movie_three')

In [ ]:
stim_table_nm.head()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
The movie is repeated 10 times. Each repeat begins when frame 0 is presented. Find the start times for each movie repeat.
</div style>

In [ ]:
start_times = stim_table_nm[stim_table_nm.frame==0].start.values

In [ ]:
start_times

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
Now we're going to average each cell's response to the movie. Start from the start time you just found above. The movie is 3600 frames long (you can check that by finding how many unique frames there are in the stimulus table). So make an array that is size (10,3600) and put the response of the cell for each of the 10 trials in the array. Then average across those 10 trials and plot the result. Do this for both of the two cells we looked at in exercises 10-12.
</div style>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
The two cells we're looking at have similar tuning curves for their drifting grating responses (see above). Do you expect they have similar responses for the natural movie? Why or why not? Compute the correlation between the averaged movie responses of these two cells. How does it compare to the correlation of their tuning to the drifting gratings? Why might they be similar? Why might they be different?
</div style>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
Do the same comparison for the second pair of cells that you looked at for the drifting gratings, cells 541511373 and 541511385.
</div style>

<div style="background: #fcdc83; border-radius: 3px; padding: 10px;">
For an extra challenge, what is the relationship between the correlation of grating responses to the correlation of movie responses across the whole population? To do this, you'll want to get the traces for all of the cells in the experiment. Look at our function at the beginning called get_dff_traces_and_stim_table(). If you use some of the code in there, and change the get_dff_traces line to:
    
> timestamps, dff = data_set.get_dff_traces()

you will get an array of the DFF traces of all of the cells. From there you can calculate tuning for all the cells and calculate the pairwise correlations. What do you expect the result to be?
    
</div style>